In [ ]:
import speech_recognition as sr
import pyttsx3
import wikipedia
import wolframalpha
import requests
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

recognizer = sr.Recognizer()
engine = pyttsx3.init()

wolfram_client = wolframalpha.Client('LJK2GV-YEYHE8PA2E')

def speak(text):
    engine.say(text)
    engine.runAndWait()

def write(text):
    print(text)
    with open("assistant_log.txt", "a", encoding="utf-8") as file:
        file.write(text + "\n")

def listen():

    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        print("Recognizing...")
        query = recognizer.recognize_google(audio, language='en-US')
        print(f"User said: {query}\n")
        return query
    except sr.UnknownValueError:
        print("Sorry, I didn't catch that. Could you please repeat?")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

def search_open_library(query):
    url = f"https://openlibrary.org/search.json?q={query}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data['numFound'] > 0:
            first_result = data['docs'][0]
            title = first_result.get('title')
            author = first_result.get('author_name', ['Unknown'])[0]
            publish_year = first_result.get('first_publish_year', 'Unknown')
            return f"Open Library: {title} by {author}, first published in {publish_year}."
        else:
            return "Open Library: Sorry, I couldn't find any relevant books."
    except requests.exceptions.RequestException as e:
        return f"Open Library: Sorry, there was an issue retrieving results: {e}"

def search_dbpedia(query):
    url = f"http://dbpedia.org/sparql"
    query_sparql = f"""
        SELECT ?abstract WHERE {{
        ?article dbo:abstract ?abstract .
        ?article rdfs:label "{query}"@en .
        FILTER (lang(?abstract) = 'en')
        }} LIMIT 1
    """
    try:
        response = requests.get(url, params={'query': query_sparql, 'format': 'json'})
        response.raise_for_status()
        data = response.json()
        if data['results']['bindings']:
            abstract = data['results']['bindings'][0]['abstract']['value']
            return f"DBpedia: {abstract}"
        else:
            return "DBpedia: Sorry, I couldn't find any relevant information."
    except requests.exceptions.RequestException as e:
        return f"DBpedia: Sorry, there was an issue retrieving results: {e}"

def process_query(query):
    if query is None:
        return

    responses = []

    try:
        res = wolfram_client.query(query)
        answer = next(res.results).text
        responses.append(f"WolframAlpha: According to WolframAlpha, {answer}")
    except Exception as e:
        responses.append(f"WolframAlpha: Sorry, I couldn't retrieve information. Error: {e}")

    try:
        query = query.replace("wikipedia", "")
        results = wikipedia.summary(query, sentences=2)
        responses.append(f"Wikipedia: According to Wikipedia, {results}")
    except wikipedia.exceptions.DisambiguationError as e:
        responses.append("Wikipedia: The query was too ambiguous. Please be more specific.")
    except wikipedia.exceptions.PageError:
        responses.append("Wikipedia: Sorry, I couldn't find anything on Wikipedia.")
    except Exception as e:
        responses.append(f"Wikipedia: Sorry, I ran into an issue. Error: {e}")

    open_library_result = search_open_library(query)
    responses.append(open_library_result)

    dbpedia_result = search_dbpedia(query)
    responses.append(dbpedia_result)

    if responses:
        for response in responses:
            write(response)
            speak(response)

if __name__ == "__main__":
    nltk.download('punkt')  
    speak("Hello, how can I assist you today?")
    while True:
        query = listen()
        if query:
            if "exit" in query.lower() or "bye" in query.lower() or "stop" in query.lower():
                speak("Goodbye!")
                break
            process_query(query)
